## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Weather
0,0,Chokurdakh,RU,70.6333,147.9167,9.21,92,25,4.65,scattered clouds
1,1,Monte Alegre,BR,-2.0078,-54.0692,71.74,97,100,1.32,light rain
2,2,Pimentel,PE,-6.8367,-79.9342,66.00,85,100,1.99,overcast clouds
3,3,Rincon,US,32.2960,-81.2354,64.40,94,1,6.91,clear sky
4,4,Comodoro Rivadavia,AR,-45.8667,-67.5000,68.00,45,0,14.97,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.

preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]

preferred_cities_df.head(10)

,City ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Weather
5,5,Saint-Philippe,RE,-21.3585,55.7679,78.80,89,75,19.57,light rain
6,6,Vaini,TO,-21.2000,-175.2000,77.00,88,40,6.91,light rain
7,7,Sola,VU,-13.8833,167.5500,82.99,74,22,10.65,moderate rain
8,8,Atuona,PF,-9.8000,-139.0333,80.01,79,27,17.31,light rain
9,9,Xichang,CN,27.8964,102.2634,82.87,23,81,14.07,broken clouds
12,12,Port Blair,IN,11.6667,92.7500,86.00,70,75,4.61,broken clouds
28,28,Bedele,ET,8.4560,36.3530,75.20,37,55,8.93,broken clouds
37,37,Rikitea,PF,-23.1203,-134.9692,77.54,75,3,5.73,clear sky
41,41,Georgetown,MY,5.4112,100.3354,89.01,66,40,8.05,scattered clouds
51,51,Victoria,HK,22.2855,114.1577,79.00,59,64,5.01,broken clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City ID            177
City               177
Country            177
Lat                177
Lng                177
Max Temp           177
Humidity           177
Cloudiness         177
Wind Speed         177
Current Weather    177
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.

# **There are no empty rows**

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Weather", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Weather,Lat,Lng,Hotel Name
5,Saint-Philippe,RE,78.80,light rain,-21.3585,55.7679,
6,Vaini,TO,77.00,light rain,-21.2000,-175.2000,
7,Sola,VU,82.99,moderate rain,-13.8833,167.5500,
8,Atuona,PF,80.01,light rain,-9.8000,-139.0333,
9,Xichang,CN,82.87,broken clouds,27.8964,102.2634,
12,Port Blair,IN,86.00,broken clouds,11.6667,92.7500,
28,Bedele,ET,75.20,broken clouds,8.4560,36.3530,
37,Rikitea,PF,77.54,clear sky,-23.1203,-134.9692,
41,Georgetown,MY,89.01,scattered clouds,5.4112,100.3354,
51,Victoria,HK,79.00,broken clouds,22.2855,114.1577,


In [10]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try: 
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except(IndexError):
        print("Hotel not found...skipping.")

Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.


In [11]:
# 7. Drop the rows where there is no Hotel Name.
#hotel_df.dropna()
hotel_df.count()

City               177
Country            177
Max Temp           177
Current Weather    177
Lat                177
Lng                177
Hotel Name         177
dtype: int64

In [12]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_Vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [15]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Weather}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# 11a. Add a marker layer for each city to the map.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

# 11b. Display the figure
fig.add_layer(marker_layer)

# Display the fig
fig